## <center> Privileged Programs and Potential Vulnerabilities </center>

### What is a privileged program?

- A program that is executable by users but carries elevated (administrative) power
- Example: `passwd`

### Where are passwords stored in Linux?

### Solution: Two-tiered Access Control Design

<center> <img src="figure/set-uid/privilege.png" width="400"/>

### Approaches to designing privilege programs
- Daemon (background processes) launched by privileged user ID (root)
- Utilize Set-UID mechanism of Linux to mark a program is privileged. 
  - The program is owned by **root** but executable by **users**

### Set-UID

- Grant privilege to the task (the program), not the user
- Possible by leveraging a process' user ID:
  - real user ID
  - effective user ID
  - saved user ID

```
$ id
$ which id
$ cp /usr/bin/id ./myid
$ ls -l ./myid
```

```
$ sudo chown root myid
$ ./myid
```

```
$ sudo chmod 4755 myid
$ ./myid
```

### Example of how Set-UID works

Using `cat` to view `/etc/shadow`

#### Switch to a terminal and be hands-on!
- Make a copy of `cat` in your home directory
- Change ownership of the copy to root
- Test to see if you can view `/etc/shadow`
- Make change to the copy of `cat` using the set-uid mechanism
- Can you view `/etc/shadow` now?

### What can go wrong?

Attack surface of set-uid

<center> <img src="figure/set-uid/attack-surface.png" width="700"/>

#### 1. User Inputs (Explicit inputs)
- Failure of input sanitization
- Buffer Overflow
- SQL Injection

#### 2. System Inputs

- Inputs provided by system but could be faked by other users
- Race Condition

#### 3. Environment Variables (Hidden inputs)

- Can be manipulated by users

#### 4. Capability Leaking
- Privileges are not properly removed

- Switch to the terminal tab
- Create a file named `zzz` in the `/etc` directory with some contents (You will need to use `sudo` or become `root` to create this file)

In [1]:
%%writefile source/cap_leak.c

/**********************************************
 * Listing 1.1: Capability leaking 
 **********************************************/
#include <stdio.h>
#include <stdlib.h>
#include <fcntl.h>

void main()
{
  int fd;
  char *v[2];

  /* Assume that /etc/zzz is an important system file,
   * and it is owned by root with permission 0644.
   * Before running this program, you should create
   * the file /etc/zzz first. */
  fd = open("/etc/zzz", O_RDWR | O_APPEND);
  if (fd == -1) {
     printf("Cannot open /etc/zzz\n");
     exit(0);
  }

  // Print out the file descriptor value
  printf("fd is %d\n", fd);

  // Permanently disable the privilege by making the 
  // effective uid the same as the real uid
  setuid(getuid());

  // Execute /bin/sh
  v[0] = "/bin/sh"; v[1] = 0;
  execve(v[0], v, 0);
} 

Overwriting source/cap_leak.c


```
$ cd ~
$ gcc -o cap_leak Computer-Security/source/cap_leak.c
$ sudo chown root cap_leak
$ sudo chmod 4755 cap_leak
$ ls -l cap_leak
```

```
$ cat /etc/zzz
$ echo 'insert bad data' >> /etc/zzz
```

```
$ ./cap_leak
$ echo 'insert really bad data' >& ...
$ exit
$ cat /etc/zzz
```  
    

### Case Example

- https://www.sektioneins.de/en/blog/15-07-07-dyld_print_to_file_lpe.html
- https://security.stackexchange.com/questions/94640/how-does-the-dyld-privilege-escalation-vulnerability-work-on-os-x

#### Invoking other programs through capability leaking: system()

- what does `system()` do?
- `man system`

In [3]:
%%writefile source/catall.c

/**********************************************
 * Code on Page 17 (Section 1.5.1)
 **********************************************/
#include <string.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[])
{
  char *cat="/bin/cat";

  if(argc < 2) {
    printf("Please type a file name.\n");
    return 1;
  }

  char *command = malloc(strlen(cat) + strlen(argv[1]) + 2);
  sprintf(command, "%s %s", cat, argv[1]);
  system(command);
  return 0 ;
}

Overwriting source/catall.c


```
$ cd ~
$ gcc -o catall Computer-Security/source/catall.c
$ sudo chown root catall
$ sudo chmod 4755 catall
$ ls -l catall
```


```
$ catall /etc/shadow
$ catall 'aa;/bin/sh'
```

The above does not work because both `bash` and `dash` shell have addressed the vulnerability in `system()`:

http://man7.org/linux/man-pages/man3/system.3.html
system() will not, in fact, work properly from programs with set-user-ID or set-group-ID privileges on systems on which /bin/sh is bash version 2: as a security measure, bash 2 drops privileges on startup.  (Debian uses a different shell, dash(1), which does not do this when invoked as sh.)

To observe the vulnerability, the following must be done:
    
```
$ sudo mv /bin/sh /bin/sh_old
$ sudo nano /bin/sh
  #!/bin/dash -p
  exec /bin/dash -p "$@"
$ sudo chmod 755 /bin/sh
```

In [4]:
%%writefile source/catallsafe.c

/**********************************************
 * Code on Page 18 (Section 1.5.2)
 **********************************************/
#include <string.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char *argv[])
{
  char *v[3];

  if(argc < 2) {
    printf("Please type a file name.\n");
    return 1;
  }

  v[0] = "/bin/cat"; v[1] = argv[1]; v[2] = 0;
  execve(v[0], v, 0);

  return 0 ;
}

Overwriting source/catallsafe.c


#### Compile and run callallsafe.c

- Do we have the same vulnerability?

### Lesson Learned
- Principle of data/code isolation
- Principle of least privilege